In [ ]:
from collections import deque
from typing import List, Tuple, Set, Dict, Optional, NewType

In [ ]:
Board = List[List[int]]
Variable = Tuple[int, int]
Domains = Dict[Variable, Set[int]] 
Assignment = Dict[Variable, int] 

In [ ]:

class SudokuSolver:

    def __init__(self, board: Board):
        self.board = board
        self.variables: List[Variable] = [(r, c) for r in range(9) for c in range(9)]

        self.domains: Domains = {}
        for r in range(9):
            for c in range(9):
                var = (r, c)
                initial_value = board[r][c]
                # Set initial possibilities for each cell
                if initial_value == 0:
                    self.domains[var] = set(range(1, 10)) # Empty cell: 1-9
                elif 1 <= initial_value <= 9:
                    self.domains[var] = {initial_value} # Pre-filled cell: only its value
                else:
                    # Handle weird input values
                    print(f"Warning: Invalid value {initial_value} at {(r,c)}. Treating as empty.")
                    self.domains[var] = set(range(1, 10))

        # Find all neighbors for each cell (row, col, box)
        self.neighbors: Dict[Variable, Set[Variable]] = {var: self._get_neighbors(var) for var in self.variables}

        # Make sure initial clues are respected in domains
        if not self.enforce_node_consistency():
             print("Warning: Initial board seems inconsistent.")


    def _get_neighbors(self, var: Variable) -> Set[Variable]:
        """Find all cells constrained by this one (same row, col, or 3x3 box)."""
        r, c = var
        neighbors = set()
        # Row
        for col_idx in range(9):
            if col_idx != c:
                neighbors.add((r, col_idx))
        # Column
        for row_idx in range(9):
            if row_idx != r:
                neighbors.add((row_idx, c))
        # 3x3 Box
        start_row, start_col = (r // 3) * 3, (c // 3) * 3
        for row_idx in range(start_row, start_row + 3):
            for col_idx in range(start_col, start_col + 3):
                neighbor_var = (row_idx, col_idx)
                if neighbor_var != var:
                    neighbors.add(neighbor_var)
        return neighbors

    # 1. enforce_node_consistency
    def enforce_node_consistency(self) -> bool:
        """Ensure initial clues are the only option in their cell's domain."""
        for var in self.variables:
            r, c = var
            initial_value = self.board[r][c]

            if initial_value != 0: # If it's a pre-filled clue
                # Check if the clue is even possible (it should be)
                if initial_value not in self.domains[var]:
                     self.domains[var] = set() # Problem! Mark as impossible.
                     return False

                # Lock domain to the initial clue
                if self.domains[var] != {initial_value}:
                    self.domains[var] = {initial_value}

        return True # Looks consistent

    # 2. revise(x, y)
    def revise(self, x: Variable, y: Variable) -> bool:
        """Make cell 'x' consistent with cell 'y'.
           Remove values from x's domain if they have no possible match in y's domain.
           Returns True if x's domain was changed.
        """
        revised = False
        values_to_remove = set()

        for val_x in self.domains[x]:
            # Can val_x work with *any* value currently possible for y?
            if not any(val_x != val_y for val_y in self.domains[y]):
                # No value in y.domain allows val_x in x.domain
                values_to_remove.add(val_x)
                revised = True

        if revised:
            self.domains[x] -= values_to_remove

        return revised

    # 3. ac3()
    def ac3(self, initial_queue: Optional[List[Tuple[Variable, Variable]]] = None) -> bool:
        """Use AC-3 algorithm to make all cells arc-consistent.
           Returns False if an inconsistency (empty domain) is found.
        """
        # Start with all pairs of neighbors, unless a specific queue is given
        if initial_queue is None:
             queue = deque([(x, y) for x in self.variables for y in self.neighbors[x]])
        else:
             queue = deque(initial_queue)

        while queue:
            x, y = queue.popleft()

            if self.revise(x, y): # If we removed values from x...
                if not self.domains[x]:
                    return False 

                # Re-check neighbors of x (because x changed)
                for z in self.neighbors[x]:
                    if z != y:
                        queue.append((z, x))

        return True # Consistent

    # 4. assignment_complete(assignment)
    def assignment_complete(self, assignment: Assignment) -> bool:
         """Check if all cells have been assigned a value."""
         return len(assignment) == len(self.variables)

    def _is_value_consistent(self, var: Variable, value: int, assignment: Assignment) -> bool:
        """Check if this value works with the assigned neighbors."""
        for neighbor in self.neighbors[var]:
            if neighbor in assignment and assignment[neighbor] == value:
                return False # Conflict!
        return True # OK with current assignments

   # 5. consistent(assignment)
    def consistent(self, assignment: Assignment) -> bool:
        for var, value in assignment.items():
            # Check against assigned neighbors
            for neighbor in self.neighbors[var]:
                if neighbor in assignment and assignment[neighbor] == value:
                    return False # there's Conflict
        return True # All assigned values are okay together

    # 6. order_domain_values(var, assignment)
    def order_domain_values(self, var: Variable, assignment: Assignment) -> List[int]:
   
        if len(self.domains[var]) == 1:
             return list(self.domains[var]) # Only one choice

        # Count how many choices each potential value removes for neighbors
        counts = {}
        for val in self.domains[var]:
            count = 0
            # Only consider values that don't immediately conflict
            if self._is_value_consistent(var, val, assignment):
                for neighbor in self.neighbors[var]:
                    # How many options does this value remove for unassigned neighbors?
                    if neighbor not in assignment and val in self.domains[neighbor]:
                        count += 1
            else:
                 count = float('inf') # This value isn't even possible right now
            counts[val] = count

        # Sort by count (smallest count first)
        sorted_values = sorted(self.domains[var], key=lambda v: counts.get(v, float('inf')))
        # Return only the currently consistent values
        return [v for v in sorted_values if counts.get(v, float('inf')) != float('inf')]


    # 7. select_unassigned_variable(assignment)
    def select_unassigned_variable(self, assignment: Assignment) -> Optional[Variable]:
        """Choose the best cell to assign next (MRV + Degree heuristic)."""
        unassigned_vars = [v for v in self.variables if v not in assignment]
        if not unassigned_vars:
            return None # All done

        # MRV: Find cells with the fewest remaining possible values
        min_domain_size = min(len(self.domains[v]) for v in unassigned_vars if self.domains[v])
        mrv_vars = [v for v in unassigned_vars if len(self.domains[v]) == min_domain_size]

        # If only one cell has the minimum, pick it
        if len(mrv_vars) == 1:
            return mrv_vars[0]

        max_degree = -1
        selected_var = None
        for v in mrv_vars:
            degree = sum(1 for n in self.neighbors[v] if n not in assignment)
            if degree > max_degree:
                max_degree = degree
                selected_var = v
        return selected_var if selected_var is not None else (mrv_vars[0] if mrv_vars else None)


    # 8. backtrack(assignment)
    def backtrack(self, assignment: Assignment) -> Optional[Assignment]:
        """Recursive backtracking search to find a solution."""

        # Base case: Are we done?
        if self.assignment_complete(assignment):
             return assignment if self.consistent(assignment) else None # Check final solution

        # Choose the next cell to try
        var = self.select_unassigned_variable(assignment)
        if var is None:
             return None # No valid variable to select, dead end

        # Try possible values for the chosen cell (ordered by LCV)
        for value in self.order_domain_values(var, assignment):

            # Check if this value conflicts with *current* assignments
            if self._is_value_consistent(var, value, assignment):
                # Looks promising, try assigning it
                new_assignment = assignment.copy()
                new_assignment[var] = value

                # Save domains before trying this path
                saved_domains = {v: dom.copy() for v, dom in self.domains.items()}

                self.domains[var] = {value} # Tentatively set domain
                inference_successful = True

                if inference_successful:
                    # Recurse...
                    result = self.backtrack(new_assignment)
                    if result is not None:
                        return result 
                    
                self.domains = saved_domains

        # If no value worked for 'var', backtrack
        return None

    def solve(self) -> Optional[Assignment]:
        """Attempt to solve the Sudoku puzzle."""
        start_time = time.time()
        print(f"Starting solver...")

        # Initial consistency checks (node is done in init)
        if not self.ac3():
            print("AC-3 failed initially. Puzzle is unsolvable.")
            return None

        # Create initial assignment from cells already solved by AC-3
        initial_assignment = {}
        for var in self.variables:
             if len(self.domains[var]) == 1:
                 value = next(iter(self.domains[var]))
                 if self._is_value_consistent(var, value, initial_assignment):
                     initial_assignment[var] = value
                 else:
                     print(f"Inconsistency found setting up initial assignment for {var}={value}.")
                     return None # Problem before backtracking even starts

        # Start the main search
        solution = self.backtrack(initial_assignment)

        end_time = time.time()
        # Get current time in Windhoek timezone
        current_time_str = time.strftime("%Y-%m-%d %I:%M:%S %p %Z") # e.g., 2025-04-30 11:40:00 PM CAT
        print(f"Search finished at {current_time_str}. Duration: {end_time - start_time:.4f} seconds.")

        if solution is None:
            print("No solution found.")
            return None
        else:
            if not self.assignment_complete(solution) or not self.consistent(solution):
                 print("Error: Solver returned an invalid solution.")
                 return None
            print("Solution found.")
            return solution


    @staticmethod
    def print_board(assignment: Optional[Assignment], board_size=9):
        """Prints the board nicely."""
        if assignment is None:
            print("No assignment to print.")
            return

        # Convert assignment dict to a 2D list for printing
        board_repr = [['.' for _ in range(board_size)] for _ in range(board_size)]
        for (r, c), val in assignment.items():
            if 0 <= r < board_size and 0 <= c < board_size:
                 board_repr[r][c] = str(val)

        box_size = int(board_size**0.5)
        for r in range(board_size):
            # Box horizontal line
            if r > 0 and r % box_size == 0:
                print("-" * (board_size * 2 + box_size - 1))
            line = []
            for c in range(board_size):
                # Box vertical line
                if c > 0 and c % box_size == 0:
                    line.append("|")
                line.append(board_repr[r][c])
            print(" ".join(line))



In [ ]:
# test
if __name__ == "__main__":
    
    board1 = [
        [0, 0, 0, 6, 0, 0, 4, 0, 0],
        [7, 0, 0, 0, 0, 3, 6, 0, 0],
        [0, 0, 0, 0, 9, 1, 0, 8, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 5, 0, 1, 8, 0, 0, 0, 3],
        [0, 0, 0, 3, 0, 6, 0, 4, 5],
        [0, 4, 0, 2, 0, 0, 0, 6, 0],
        [9, 0, 3, 0, 0, 0, 0, 1, 0],
        [0, 2, 0, 0, 0, 0, 0, 0, 0]
    ]

    

    print("Solving Board 1:")
    solver1 = SudokuSolver(board1)
    solution1 = solver1.solve()
    SudokuSolver.print_board(solution1)
